# Coronavirus: Loneliness

An irregular shaped spreadsheet dataset originally published by the UK Office for National Statistics.

## Tutorial Structure

With these example tutorials I'm going to comment heavily and cover nuances in a follow up section (with liberal targetted previews as needed) as it's the easiest way to grapple with new ideas. It may also be worth opening up these notebooks yourself (they're in `./jupyterbook` in the [tidychef](https://github.com/mikeAdamss/tidychef) github repo) so you can run, alter and generally have a play about with this yourself as part of your learning.

We'll cover:

- source data
- requirements, what we're aiming to do here
- show the full script (all logic commented)
- output the selection preview
- nuances (where applicable)
- view the output

This sequencing is necessary as the output for some of the example is **really** long so that necessitates it coming last. If you're viewing this via a jupyter book (i.e on the site) you can navigate between the above sections via your right hand menu.

_Note - these tutorial scripts might seem verbose due all the comments but that's ok (this is a tutorial after all). If you take them out you end up with a fairly succinct and human readable encapsulation of what would otherwise (with existing tools) be a rather convoluted and fragile set of instructions to express._

In virtually all cases I'll make heavy use of `preview` and `bounded` to only look at relevant parts of what can be quite large datasets. Downloads links are provided for the source data.

## Source

In [ ]:
from tidychef import acquire, preview
from tidychef.selection import XlsxSelectable

table: XlsxSelectable = acquire.xlsx.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/coronavirusandlonelinessreferencetable.xlsx", tables="1.4 Loneliness by sex")
preview(table, bounded="A1:F24")

From an xlsx source which can be [downloaded here](https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/coronavirusandlonelinessreferencetable.xlsx).

## Requirements


- We'll take the priciple description of lonliness (Often/Always etc) as a "Lonliness" column.
- We'll take sex as a "Sex" column.
- Sample size will be taken as a "Sample Size" column.
- We'll add a unit of measure so our data makes sense.

One the whole thiis is a particularly good example or a _very_ irregularly presented table, in terms of readibility this was clearly very much constructed for a human not a machine to read.

In [ ]:
from tidychef import acquire, preview
from tidychef.direction import right, down, left, up
from tidychef.output import TidyData, Column
from tidychef.selection import XlsxSelectable

# Get the data via the acquire api
table: XlsxSelectable = acquire.xlsx.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xlsx/coronavirusandlonelinessreferencetable.xlsx", tables="1.4 Loneliness by sex")

# Lets find the unwanted footer information so we can dicount it where needed
# - (a) select the cell with the text "Source: " in it
# - (b) confirm there's only one of those
# - (c) expand to a box selection (think "hold clicking a cell and dragging down right" in excel)
unwanted_footer = table.cell_containing_string("Source: ", strict=False).expand_to_box()
table = table - unwanted_footer

# - (a) get the column with Male in it
# - (b) narrow it down to just specifc cells from that column with Male in them
# - (c) extend/extrude the selection to inlcude two cells to the right of each
sex = table.column_containing_strings(["Male"]).re("Male").extrude(right(2)).label_as("Sex")

# - (a) filter the data to cells with the string "Sample size" in them
# - (b) confirm there's just two of those
# - (c) select all cells located to the right of those two
# - (d) desregard any blank cells you just selected
sample_size = table.cells_containing_string("Sample size", strict=False).assert_len(2).fill(right).is_not_blank().label_as("Sample Size")

# For observations, we just want all numeric values that aren't the sample size
observations = table.is_numeric().label_as("Observations") - sample_size

# Loneliness is anything to the left of an observation cell thats not a number
loneliness = observations.fill(left).is_not_numeric()

# Create a bounded preview inline but also write the full preview to path
preview(observations, sex, sample_size, loneliness, bounded="A1:F20")
preview(observations, sex, sample_size, loneliness, path="loneliness.html")

# Construct the tidydata table
tidy_data = TidyData(
    observations, 
    Column(sex.attach_directly(down)),
    Column(sample_size.attach_directly(up)),
    Column(loneliness.attach_directly(right)),
    Column.constant("Unit of measure", "Percentage")
)

tidy_data.to_csv("loneliness.csv")

## Nuances

### Subtracting junk text

There's a nice example here if why tidychef supports the **substraction of one selection from another**. Human first tables have a nasty habit of including all sorts of ad hoc footers and side comments that don't fit into any sort of regular structured data view of a resource.

```python
unwanted_footer = table.filter(filters.contains_string("Source: ")).assert_one().expand_to_box()
```

which allows us to casually disregard the junk text later on as needed, i.e

```python
loneliness = loneliness - unwanted_footer # clear it up a bit
```

### Sex - why the double 'Male' selection?

This probably seems odd at first. The key point here is **re is slow**.

So this would work:

```python
sex = table.re("Male").extrude(right(2)).label_as("Sex")

```

but this is smarter

```python
sex = table.column_containing_strings(["Male"]).re("Male").extrude(right(2)).label_as("Sex")

```

...in complete honestly, "slow" is relative and unless you're running `re` against 10's of thousands of cells it's probably fine, even then its minutes not hours -neverthelss- it _is_ a lot slower than the other standard methods so I wanted to make the point. It's good practice to always aim to tighten your selection _prior_ to `re` where it's viable to.


# Output

The full preview can be [viewed here](./loneliness.html).

The tidy data can be [downloaded here](./loneliness.html) and a full inline preview of the tidydata generated is shown below.

In [ ]:
print(tidy_data)